In [1]:
import pandas as pd
import numpy as np
import regex as re

In [2]:
dataset = pd.read_csv('data/data_clean_removed.csv')
dataset.drop(["Unnamed: 0"],inplace= True,axis = 1)

In [3]:
dataset = dataset.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))


In [5]:
dataset.replace({'elevatoe': {1: 'jest', 0: 'nie ma'}}, inplace=True)
dataset.replace({'Parkingplace': {1: 'jest', 0: 'nie ma'}}, inplace=True)
dataset.replace({'balkon': {1: 'jest', 0: 'nie ma'}}, inplace=True)
dataset.replace({'taras': {1: 'jest', 0: 'nie ma'}}, inplace=True)
dataset.replace({'ogrdek': {1: 'jest', 0: 'nie ma'}}, inplace=True)

In [6]:
dataset.head(4)

,Price,Space,Rooms,Market,Year,elevator,Parkingplace,balkon,taras,ogrdek,district,street,level,max_level
0,4000000,560,4,wtórny,2015,1,jest,jest,jest,jest,Wawer,Patriotów,3,3
1,4000000,560,4,wtórny,2015,1,jest,jest,jest,jest,Wawer,Patriotów,3,3
2,21350100,487,10,wtórny,2021,1,jest,nie ma,nie ma,nie ma,Śródmieście,Elektryczna,6,6
3,19900000,487,6,wtórny,2001,1,jest,nie ma,nie ma,nie ma,Mokotów,Cybulskiego,5,5


In [7]:
dataset_dummies = pd.get_dummies(dataset,columns=['Market','street','district'])

In [8]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error


x_train,x_test,y_train,y_test = train_test_split(dataset_dummies.drop(['Price'],axis=1),dataset_dummies['Price'],test_size=0.2,random_state=42)

In [9]:
gbr  = GradientBoostingRegressor()
params = {'n_estimators':[100,200,300,400,500,600,700,800,900,1000],'max_depth':[1,2,3,4,5,6,7,8,9,10],'learning_rate':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]}
search = RandomizedSearchCV(gbr,param_distributions=params,cv=5,return_train_score=False)

search.fit(x_train,y_train)

c:\Users\ja\anaconda3\envs\data_science\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
50 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
50 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\ja\anaconda3\envs\data_science\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\ja\anaconda3\envs\data_science\lib\site-packages\sklearn\ensemble\_gb.py", line 486, in fit
    X, y = self._validate_data(
  File "c:\Users\ja\anaconda3\envs\data_science\lib\site-packages\sklearn\base.py", line 581, in _validate_data
    X, y = check_X_y(X, y, 

ValueError: could not convert string to float: 'jest'

In [ ]:
search.best_estimator_

In [ ]:
import pickle

filename = 'gbr_model.sav'
pickle.dump(best_gbr, open(filename, 'wb'))